<a href="https://colab.research.google.com/github/MisterGusMan/Deep_Learning_IGTI/blob/main/M%C3%B3dulo%202/Trabalho_Pr%C3%A1tico_M%C3%B3dulo_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import tensorflow.keras.models as Models
from tensorflow.keras.models import Model
import tensorflow.keras.layers as Layers
import tensorflow.keras.optimizers as Optimizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plot
import matplotlib
import numpy as np
import cv2
from random import randint
from sklearn.metrics import confusion_matrix
import seaborn as sn
import os

In [52]:
def get_images_labels(directory):
  Images = []
  Labels = []
  label = 0
  for labels in os.listdir(directory): # diretorios das imagens que possuem nomes das classes.
    if labels == 'glacier':
      label = 2
    elif labels == 'sea':
      label = 4
    elif labels == 'buildings':
      label = 0
    elif labels == 'forest':
      label = 1
    elif labels == 'street':
      label = 5
    elif labels == 'mountain':
      label = 3
    for image_file in os.listdir(directory+labels): # extrai imagens
      image = cv2.imread(directory+labels+r'/'+image_file) # lê imagens (OpenCV)
      image = cv2.resize(image, (150,150)) # "resize" imagens
      Images.append(image)
      Labels.append(label)
  return (Images, Labels)

In [53]:
def get_images(directory):
  Images = []
  for image_file in os.listdir(directory): # extrai imagens
    image = cv2.imread(directory+'/'+image_file) # lê imagens (OpenCV)
    image = cv2.resize(image, (150, 150)) # "resize" imagens
    Images.append(image)
  return (Images)

In [54]:
def get_classlabel(class_code):
  labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3: 'mountain'}
  return labels[class_code]


In [55]:
def plot_images():
  Images, Labels = get_images_labels('input/seg_train/seg_train/') # extrai imagens de treinamento
  Images = np.array(Images) # converte lista de imagens para numpy array.
  Labels = np.array(Labels)
  f, ax = plot.subplots (5,5)
  for i in range(0,5,1):
    for j in range(0,5,1):
      index = randint(0, len(Images))
      ax[i,j].imshow( Images[index])
      ax[i,j].set_title(get_classlabel(Labels[index]))
      ax[i,j].axis('off')
      plot.show()


In [56]:
def pie_plot():
  class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
  Images, Labels = get_images_labels ('input/seg_train/seg_train/')
  Labels = np.array(Labels)
  sizes = np.bincount(Labels)
  explode = (0, 0, 0, 0, 0, 0)
  plot.pie(sizes, explode=explode, labels=class_names, autopct='%1.1f%%', shadow=True, startangle = -150)
  plot.axis ('equal')
  plot.title('Porcentagem de Imagens por Categoria')
  plot.show()


In [57]:
def get_model_Sequential_Completo():
  model = Models. Sequential()
  model.add(Layers.Conv2D(200, kernel_size=(3,3), activation='relu', input_shape=(150, 150,3)))
  model.add(Layers.Conv2D(180, kernel_size=(3,3), activation='relu'))
  model.add(Layers. MaxPool2D(5,5))
  model.add(Layers.Conv2D(180, kernel_size=(3,3), activation='relu'))
  model.add(Layers.Conv2D(140, kernel_size=(3,3), activation='relu'))
  model.add(Layers.Conv2D(100, kernel_size=(3,3), activation='relu'))
  model.add(Layers.Conv2D(50, kernel_size=(3,3), activation='relu'))
  model.add(Layers. MaxPool2D(5,5))
  model.add(Layers. Flatten())
  model.add(Layers.Dense(180, activation='relu'))
  model.add(Layers.Dense(100, activation='relu'))
  model.add(Layers.Dense(50, activation='relu'))
  model.add(Layers.Dropout(rate=0.5))
  model.add(Layers.Dense(6, activation='softmax'))
  model.compile(optimizer=Optimizer. Adam(lr=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  model. summary()
  return model


In [58]:
def plot_Model_Sequential_Completo():
  model = get_model_Sequential_Completo()
  plot_model(model, to_file='modelo_completo.png', show_shapes=True)
  model_image = cv2.imread('modelo_completo.png')
  model_image_np = np.array(model_image)
  f, ax = plot. subplots(1,1)
  ax.imshow(model_image_np)
  ax.set_title('Modelo Completo')
  ax.axis('off')
  plot.show()


In [59]:
def fit_model_Sequential_Completo():
  Images, Labels = get_images_labels('input/seg_train/seg_train/')
  Images = np.array(Images) # converte lista de imagens para numpy array.
  Labels = np.array(Labels)
  np.random.shuffle (Images)
  np.random.shuffle(Labels)
  few_images - Images [1:300]
  few_labels = Labels[1:300]
  model = get_model_Sequential_Completo()
  trained = model.fit(few_images, few_labels, batch_size=10, epochs=4, validation_split=0.30)
  model.save('model/first')
  model.save_weights('model/first')
  plot.plot(trained.history['accuracy'])
  plot.plot(trained.history['val_accuracy'])
  plot.title('Model accuracy')
  plot.ylabel('Accuracy')
  plot.xlabel('Epoch')
  plot.legend ( ['Train', 'Test'], loc='upper left')
  plot.show()
  plot.plot(trained.history['loss'])
  plot.plot(trained.history['val_loss'])
  plot.title('Model loss')
  plot.ylabel('Loss')
  plot.xlabel( 'Epoch')
  plot.legend(['Train', 'Test'], loc='upper left')
  plot.show()


In [60]:
def predict_model_Sequential_Completo():
  model = get_model_Sequential_Completo()
  model.load_weights('model/first')
  pred_images = get_images('input/seg_pred/seg_pred/')
  pred_images = np.array(pred_images)
  fig = plot.figure(figsize=(60, 60))
  outer = matplotlib.gridspec.GridSpec(3, 3, wspace=0.4, hspace=0.4)
  for i in range (9):
    inner = matplotlib.gridspec. GridSpecFromSubplotSpec(2, 1, subplot_spec-outer[i], wspace=0.1, hspace=0.1)
    rnd_number = randint(0, len(pred_images))
    pred_image = np.array([pred_images[rnd_number]])
    predict_classes = model.predict(pred_image)
    arg = np.argmax(predict_classes, axis=1)[0]
    pred_class = get_classlabel(arg)
    pred_prob = predict_classes.reshape(6)
    for j in range(2):
      if (j%2) == 0 :
        ax = plot. Subplot(fig, inner[j])
        ax.imshow(pred_image[0])
        ax.set_title(pred_class)
        ax.set_xticks([])
        ax.set_yticks([])
        fig.add_subplot(ax)
      else:
        ax = plot. Subplot(fig, inner[j])
        ax.bar([get_classlabel(0), get_classlabel(1),get_classlabel(2),get_classlabel(3), get_classlabel(4), get_classlabel(5)], pred_prob)
        fig. add_subplot(ax)
  plot.show()


In [61]:
def evaluate_Model_Sequential_Completo():
  test_images, test_labels = get_images_labels('input/seg_test/seg_test/')
  test_images = np.array(test_images)
  test_labels = np.array(test_labels)
  test_images = test_images[1:100]
  test_labels = test_labels[1:100]
  model = get_model_Sequential_Completo()
  model.load_weights('model/first')
  model.evaluate(test_images, test_labels, verbose=1)


In [62]:
def plot_confusion_matrix():
  class_names = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
  model = get_model_Sequential_Completo ( )
  model.load_weights('model/first')
  test_images, test_labels = get_images_labels ('input/seg_test/seg_test/')
  test_images = np.array(test_images)
  test_labels = np.array(test_labels)
  few_images = test_images
  few_labels = test_labels
  test_prob = model.predict(few_images)
  pred_labels = np.argmax(test_prob, axis = 1)
  CM = confusion_matrix(few_labels, pred_labels)
  ax = plot.axes()
  sn.set(font_scale=1.4)
  sn.heatmap(CM, annot=False, annot_kws={"size": 16}, xticklabels=class_names, yticklabels=class_names, ax = ax)
  ax.set_title('Confusion matrix')
  plot.show()
